In [1]:
from pathlib import Path
BASE_DIR = Path("/Users/dsaluru/Desktop/GPU/bird-sql-forked/save_data/qwen3/")

GENERATION_DIR = BASE_DIR / "generations"
RESULTS_DIR = BASE_DIR / "with_results"
REWARD_DIR = BASE_DIR / "with_rewards"

### Generations

In [2]:
# load the generations jsonl file - data_gen.jsonl
import json

final_generations_data = {}
with open(GENERATION_DIR / "data_gen.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)

        if data['question_id'] not in final_generations_data:
            final_generations_data[data['question_id']] = {
                'question_id' : data['question_id'],
                'db_id' : data['db_id'],
                'question' : data['question'],
                'evidence' : data['evidence'],
                'SQL' : data['SQL'],
                'difficulty' : data['difficulty'],
                'responses' : []
            }
        curr_responses = []
        for item in data['responses']:
            curr_responses.append(data['responses'][item]['content'])
        final_generations_data[data['question_id']]['responses'] += curr_responses




In [3]:
mean_responses = 0
for key in final_generations_data:
    mean_responses += len(final_generations_data[key]['responses'])
mean_responses /= len(final_generations_data)
print(mean_responses)


95.73


In [4]:
import pandas as pd
pd.Series(final_generations_data[1473]['responses']).value_counts()

SELECT AVG(Consumption) / 12.0 AS average_monthly_consumption\nFROM yearmonth AS Y\nJOIN customers AS C ON Y.CustomerID = C.CustomerID\nWHERE C.Segment = 'SME'\nAND Y.Date BETWEEN '201301' AND '201312'                                                                 10
SELECT AVG(Consumption) / 12.0 AS average_monthly_consumption\nFROM yearmonth AS Y\nJOIN customers AS C ON Y.CustomerID = C.CustomerID\nWHERE C.Segment = 'SME'\n  AND Y.Date BETWEEN '201301' AND '201312'                                                                7
SELECT AVG(Consumption) / 12.0 AS average_monthly_consumption\nFROM yearmonth AS y\nJOIN customers AS c ON y.CustomerID = c.CustomerID\nWHERE c.Segment = 'SME'\n  AND y.Date BETWEEN '201301' AND '201312'                                                                6
SELECT AVG(Consumption) / 12.0 AS average_monthly_consumption\nFROM yearmonth AS y\nJOIN customers AS c ON y.CustomerID = c.CustomerID\nWHERE c.Segment = 'SME'\nAND y.Date BETWEEN '201301' AND 

### With Results

In [5]:
final_results_data = {}
with open(RESULTS_DIR / "data_with_results.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)

        if data['question_id'] not in final_results_data:
            final_results_data[data['question_id']] = {
                'question_id' : data['question_id'],
                'db_id' : data['db_id'],
                'question' : data['question'],
                'evidence' : data['evidence'],
                'SQL' : data['SQL'],
                'difficulty' : data['difficulty'],
                'responses' : []
            }
        curr_responses = []
        for item in data['responses']:
            curr_responses.append((data['responses'][item]['content'], data['responses'][item]['evaluation']))
        final_results_data[data['question_id']]['responses'] += curr_responses

In [6]:
mean_responses = 0
for key in final_results_data:
    mean_responses += len(final_results_data[key]['responses'])
mean_responses /= len(final_results_data)
print(mean_responses)


52.02


In [7]:
import pandas as pd
mean_pass = 0
for key in final_results_data:
    try:
        mean_pass += pd.Series([x[1].split(':')[0] for x in final_results_data[key]['responses']]).value_counts()['pass']
    except:
        print("key : ", key)
mean_pass /= len(final_results_data)
print(mean_pass)

key :  532
key :  701
key :  704
key :  518
42.83


### with rewards

In [8]:
final_rewards_data = {}
with open(REWARD_DIR / "data_reward_all.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)

        if data['question_id'] not in final_rewards_data:
            final_rewards_data[data['question_id']] = {
                'question_id' : data['question_id'],
                'db_id' : data['db_id'],
                'question' : data['question'],
                'evidence' : data['evidence'],
                'SQL' : data['SQL'],
                'difficulty' : data['difficulty'],
                'responses' : []
            }
        curr_responses = []
        for item in data['responses']:
            if data['responses'][item]['evaluation'].split(':')[0] == 'pass' and data['responses'][item]['reward'] != None:
                curr_responses.append((data['responses'][item]['content'], data['responses'][item]['evaluation'], data['responses'][item]['reward']))
        final_rewards_data[data['question_id']]['responses'] += curr_responses

In [9]:
mean_responses = 0
for key in final_rewards_data:
    mean_responses += len(final_rewards_data[key]['responses'])
mean_responses /= len(final_rewards_data)
print(mean_responses)


39.06


In [10]:
# sort the responses by reward

for key in final_rewards_data:
    final_rewards_data[key]['responses'].sort(key=lambda x: x[2], reverse=True)


In [11]:

for k in range(1, 20):
    accuracy = 0
    for key in final_rewards_data:
        all_candidates_rewards = [x[1] for x in final_rewards_data[key]['responses']]
        top_k_candidates_rewards = all_candidates_rewards[:k]
        if len(final_rewards_data[key]['responses']) > 0:
            if 'pass: correct' in top_k_candidates_rewards:
                accuracy += 1
    accuracy /= len(final_rewards_data)
    print("accuracy for k = ", k, " is ", accuracy)


accuracy for k =  1  is  0.682
accuracy for k =  2  is  0.702
accuracy for k =  3  is  0.712
accuracy for k =  4  is  0.714
accuracy for k =  5  is  0.716
accuracy for k =  6  is  0.72
accuracy for k =  7  is  0.724
accuracy for k =  8  is  0.724
accuracy for k =  9  is  0.726
accuracy for k =  10  is  0.728
accuracy for k =  11  is  0.728
accuracy for k =  12  is  0.73
accuracy for k =  13  is  0.734
accuracy for k =  14  is  0.738
accuracy for k =  15  is  0.744
accuracy for k =  16  is  0.744
accuracy for k =  17  is  0.744
accuracy for k =  18  is  0.744
accuracy for k =  19  is  0.744
